In [1]:
import sys
sys.path.append("..")
sys.path.append("/home/VCTK-Corpus/wav48/p225")


In [2]:
import os
from models.segmentation_model import SegmentationModel
import tensorflow as tf
import numpy as np
from scipy.io import wavfile
import fnmatch
import librosa
import itertools 
import re 

# Define parametes

In [3]:
train_parameters = {
    "lr": 0.001,
    "decay_steps": 400,
    "decay_rate": 0.95,
    "dropout_prob": 0.05
}

model_parameters = {
    "speaker_embedding_size": 16,#
    "num_conv_layers": 2,
    "conv_num_filters": 8,
    "conv_kernel_size": [2, 2],
    "num_bidirectional_units":16 ,#16
    "num_bidirectional_layers": 2,
}

output_vocab_size=5626 #3, 29 
num_speakers = 10 #10
num_steps = 2
save_energy = 2
n_mels = 20
num_beams = 2

# Train model

In [4]:

def find_files(directory, pattern='*.wav'):
    '''Recursively finds all files matching the pattern.'''
    files = []
    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, pattern):
            files.append(os.path.join(root, filename))
    return files


files=find_files("/home/VCTK-Corpus/wav48/2wfiles")
#files=find_files("/home/VCTK-Corpus/wav48/p225")
melM=[]

from scipy.io import wavfile
for file in files:
#    print file
    
    #sample_rate, X = wavfile.read(file)
    X, sr = librosa.load(file,sr=50500)
    melM.append(librosa.feature.mfcc(X,sr=100,n_mfcc=20))
%store -r files_len
#231
#files_len=files_len
files_len=2
print melM
print len(melM[0])
print len(melM[0][0])
file_len = np.array([len(melM[i][0]) for i in range(files_len)], dtype=np.int32)
max_file_len = np.amax(file_len)
#print max_file_len
melM = [[list(np.pad(np.array(melM[i][j], dtype=np.float32),
                     (0, max_file_len - file_len[i]), 
                     'constant', constant_values=(0, 0)))
         for j in range(20)] for i in range(files_len)]
melM=np.array(melM).transpose(0,2,1)


with tf.Session() as sess:

    model = SegmentationModel(
        output_vocab_size, num_speakers, model_parameters,
    )
    
    frequencies = tf.placeholder(tf.float32, [None, None, n_mels])
    frequencies_seq_len = tf.placeholder(tf.int32, [None])
    speaker_ids = tf.placeholder(tf.int32, [None])
    phonemes = tf.sparse_placeholder(tf.int32)
    
    
    train_op_tf, loss_tf, global_step_tf, summary_tf = model.build_train_operations(
        frequencies, frequencies_seq_len,
        speaker_ids,
        phonemes,
        train_parameters
    )
    
    predictor_frequencies = tf.placeholder(tf.float32, [None, None, n_mels])
    predictor_frequencies_seq_len = tf.placeholder(tf.int32, [None])
    predictor_speaker_ids = tf.placeholder(tf.int32, [None])
    
    greedy_predictor_tf = model.build_greedy_predictor(
        predictor_frequencies, predictor_frequencies_seq_len, predictor_speaker_ids, True
    )
    
    encoder_output,beam_predictor_tf = model.build_beam_search_predictor(
        predictor_frequencies, predictor_frequencies_seq_len, predictor_speaker_ids,
        num_beams, True
    )

    train_writer = tf.summary.FileWriter('../log/train_grapheme_to_phoneme_model_notebook/train', sess.graph)
    
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    saver = tf.train.Saver(max_to_keep=3, keep_checkpoint_every_n_hours=3)
    %store -r target_indices
    %store -r target_values
    %store -r dense_shape
    
    print np.shape(melM)
    print np.shape(np.array([file_len[i]for i in range(files_len)],dtype=np.float))
    print np.shape(files_len)
    #print np.shape()
    train=1
    if train==0:
        for _ in xrange(num_steps):

    #         target_indices=[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2]]
    #         target_values=[1, 2, 3, 4, 5, 6]
    #         dense_shape=[2, 4
    #       target_indices=[[0,0],[0,1],[0,2],[1,0],[1,1],[1,2]]
    #       target_values=[1,2,3,4,5,6]
    #       dense_shape=[2,3]
            

            out = sess.run([
                train_op_tf,
                global_step_tf,
                loss_tf,
                summary_tf
            ], feed_dict={
                frequencies:melM,#231,1232,20
                frequencies_seq_len: np.array([file_len[i] for i in range(files_len)],dtype=np.float32),#1232*np.ones(231),
                speaker_ids: 1 * np.ones(files_len),
                phonemes:(target_indices,target_values, dense_shape)
            })
            #17* np.ones(1),
            #k个文件speaker的编号
    # feed_dict={
    #             frequencies: np.random.rand(2, 10, 20),
    #             frequencies_seq_len: 8* np.ones(2),
    #             speaker_ids: 2 * np.ones((2)),
    #             phonemes:(target_indices,target_values, dense_shape)
    #         })
            _, global_step, loss, summary  = out

            print global_step
            print loss

            train_writer.add_summary(summary, global_step)

            # detect gradient explosion
            if loss > 1e8 and global_step > 500:
                print('loss exploded')
                break

            if global_step % save_energy == 0 and global_step != 0:

                print('saving weights')
                if not os.path.exists('../weights/train_grapheme_to_phoneme_model_notebook'):
                    os.makedirs('../weights/train_grapheme_to_phoneme_model_notebook')
                saver.save(sess, '../weights/train_grapheme_to_phoneme_model_notebook/model2.ckpt', global_step=global_step)
    
        coord.request_stop()
        coord.join(threads)
    else:
        saver.restore(sess, '../weights/train_grapheme_to_phoneme_model_notebook/model2.ckpt-2')
        graph = tf.get_default_graph()
        
#         beam_predictor_tf = model.build_beam_search_predictor(
#         predictor_frequencies, predictor_frequencies_seq_len, predictor_speaker_ids,
#         num_beams, True
#     )
        
        encoder_output,beam_predictor=sess.run([encoder_output,beam_predictor_tf],feed_dict={predictor_frequencies:melM,
                                                               predictor_frequencies_seq_len:np.array([file_len[i]for i in range(files_len)],dtype=np.float32),
                                                               predictor_speaker_ids:2 * np.ones((files_len)),
                                                               
                                                               })
        ###############
        ###############
        q=encoder_output
        beamsize=3
        x,y,z=np.shape(encoder_output)
        #print x,y,z

        index3=[[[0 for i in range(beamsize)] for j in range(y)]for k in range(x)]
        BEAM3=[[[0 for i in range(beamsize)] for j in range(y)]for k in range(x)]
        #print np.shape(index3)
        #p=np.array([[5,2,4,6],[3,7,4,5],[9,1,10,3]])
        #p=np.random.rand(3,4)
        for t in range(x):
            #print "t is:",t
            p=q[t]
            b,c=y,z

            index=[[0 for i in range(beamsize)] for j in range(b)]
            mask=[[0 for i in range(beamsize)] for j in range(b)]
            index2=[[0 for i in range(beamsize)] for j in range(b)]

            states_1=p[0,:]#embedding_size
            index_states_1=np.argsort(states_1)
            BEAM=[]
            BEAM2=[[0 for i in range(beamsize)] for j in range(b)]#b,beamsize
            for i in range(beamsize):
                BEAM.append(states_1[index_states_1[len(states_1)-1-i]])#(beamsize)

                index[0][i]=index_states_1[len(states_1)-1-i]




            for i in range(1,b):#b:max_seq_len
                SET=p[i,:]#(c,)
                sum=[]
                for j in range(c):#c:embedding_size
                    for m in BEAM:
                        sum.append(SET[j]+m)


                sort_sum=np.argsort(sum)
                kk=0
                for k in range(len(sum)):
                    #print "i is",i,"k is",k

                    if kk==beamsize:
                        break
                    if k==0:
                        BEAM=[]
                    #print "len(sum)",len(sum),"k",k,"shape_sort_sum",np.shape(sort_sum)
                    index_sum=sort_sum[len(sum)-1-k]

                    aa=(index_sum+1)/beamsize#5 6/2=3
                    bb=(index_sum+1)%beamsize
                    if bb!=0:

                        if aa!=30 and index[i-1][bb-1]!=30 and abs(index[i-1][bb-1]-aa)>1:
                            continue
                        else:
                            index[i][kk]=aa
                            mask[i][kk]=bb-1
                            kk=kk+1
                            BEAM.append(sum[sort_sum[len(sum)-1-k]])
                            #print "k:",k,"kk:",kk,"shape_BEAM:",np.shape(BEAM)
                    if bb==0:
                        if (aa-1)!=30 and index[i-1][beamsize-1]!=30 and abs(index[i-1][beamsize-1]-aa+1)>1:
                            continue
                        else:
                            index[i][kk]=aa-1
                            mask[i][kk]=beamsize-1
                            kk=kk+1
                            BEAM.append(sum[sort_sum[len(sum)-1-k]])
                            #print "k:",k,"kk:",kk,"shape_BEAM:",np.shape(BEAM)

                #print "shape_BEAM:",np.shape(BEAM)


                BEAM2[i]=BEAM
                #print "shape_BEAM2:",np.shape(BEAM2)





                #print "BEAM2 is",BEAM2#scores

            #print index,"index_shape:",np.shape(index)

            index2=index
            for k in range(beamsize): 
                index2[b-2][k]=index[b-2][mask[b-1][k]]
                mask[b-2][k]=mask[b-2][mask[b-1][k]]
            for i in range(b-2):
                for k in range(beamsize):
                    index2[b-3-i][k]=index[b-3-i][mask[b-2-i][k]]
                    mask[b-3-i][k]=mask[b-3-i][mask[b-1-i][k]]
            #print "index2 is",index2#path

            index3[t]=index2
            BEAM3[t]=BEAM2
        yinsu_size=75
        #print "index3 is",index3,"BEAM3 is",BEAM3
        #print "shape_index3:",np.shape(index3),"shape_BEAM3:",np.shape(BEAM3)
        index_final=np.array(index3)[:,:,0]
        BEAM_final=np.array(BEAM3)[:,:,0]
        #add
        index_model4=index_final
        voiced_target=[[0 for j in range(y)]for i in range(x)]
        for i in range(x):
            for j in range(y):
                if index_model4[i][j]==z-1:
                    index_model4[i][j]=0
                else:
                    #index_model4[i][j] += 1
                    index_model4[i][j]=(((index_final[i][j]+1)/yinsu_size)+1)
                    voiced_target[i][j]=1
            
        print "index_model4:",np.shape(index_model4),index_model4
        #print "voiced_target:",type(voiced_target),voiced_target



        #add end
        #print index_final,np.shape(index_final)
        yinsudui=[[]for t in range(x)]
        duration=[[]for t in range(x)]
        len_yinsudui=[[]for t in range(x)]
        for t in range(x):
            yinsudui[t].append(index_final[t][0])
        d=1
        for t in range(x):
            for j in range(1,y):

                if index_final[t][j]!=index_final[t][j-1]:

                    yinsudui[t].append(index_final[t][j])
                    duration[t].append(d)
                    d=1
                else:
                    d=d+1
            duration[t].append(d)

        for t in range(x):
            len_yinsudui[t]=len(yinsudui[t])
        for t in range(x):
            for j in range(len_yinsudui[t]):
                if yinsudui[t][j]==z-1:
                    yinsudui[t].remove(z-1)
                    duration[t].remove(duration[t][j])
        #print "duration is",duration,"shape_duration is",np.shape(duration)
        #print "yinsudui is",yinsudui,"shape_yinsudui is",np.shape(yinsudui)
        #print "target_values is",target_values,"shape_target_values is",np.shape(target_values)       
        leijia=0
        
#         for t in range(x):
#             for j in range(len_yinsudui[t]):
#                 print "len_yinsudui[t]",len_yinsudui[t]
#                 yinsudui[t][j]=np.int(yinsudui[t][j])
                
#                 print leijia,yinsudui[t][j]
#                 print len(target_values)
#                 yinsudui2[t].append(target_values[leijia+yinsudui[t][j]])
            
#             leijia=leijia+len_yinsudui[t]-1
#         print "yinsudui2 is",yinsudui2,"shape_yinsudui2 is",np.shape(yinsudui2)
        
        #yinsudui_size=yinsu_size*yinsu_size
        #
        yinsu=[[]for t in range(x)]
        yinsu_seq_len=[0 for t in range(x)]
        for t in range(x):
            for j in range(len_yinsudui[t]):
                yinsu[t].append((yinsudui[t][j]/yinsu_size)+1)
            yinsu[t].append(yinsudui[t][len_yinsudui[t]-1]%yinsu_size)
            yinsu_seq_len[t]=len(yinsu[t])
            
        #print "yinsu is",yinsu,"shape_yinsu is",np.shape(yinsu)
        #print "yinsu_seq_len is",yinsu_seq_len

        #
        duration2=[[]for t in range(x)]
        for t in range(x):
            duration2[t].append(duration[t][0]/2)
        for t in range(x):
            for j in range(1,len_yinsudui[t]):
                duration2[t].append((duration[t][j]+duration[t][j-1])/2)
        for t in range(x):
            duration2[t].append(duration[t][len_yinsudui[t]-1]/2)
        #print "duration2 is",duration2,"shape_duration2 is",np.shape(duration2) 
        # duration=[[0 for i in range(z-1)]for t in range(x)]
        # for t in range(x):
        #     for i in range(z-1):
        #         for j in range(y):
        #             if index_final[t][j]==i:
        #                 duration[t][i]=duration[t][i]+1
        # print "duration is",duration,"shape_duration is",np.shape(duration)
        num_buckets=10
        #mean_duration=np.mean(duration,1)
        max_duration2_pre=[0 for t in range(x)]
        min_duration2_pre=[0 for t in range(x)]
        for t in range(x):
            max_duration2_pre[t]=np.max(duration2[t])
            min_duration2_pre[t]=np.min(duration2[t])
        max_duration2=np.max(max_duration2_pre)
        min_duration2=np.min(min_duration2_pre)
        #print max_duration2,min_duration2
        buckets=[0 for i in range(num_buckets)]
        
        interval=(max_duration2-min_duration2)/num_buckets
        for i in range(num_buckets):
            buckets[i]=buckets[i]+i*interval
        #print "buckets is",buckets,"shape_buckets is",np.shape(buckets)
        len_yinsu=[0 for t in range(x)]
        for t in range(x):
            len_yinsu[t]=len(yinsu[t])
        duration_buckets=[[0 for i in range(len_yinsu[t])]for t in range(x)]
        for t in range(x):
            for i in range(len_yinsu[t]):
                for k in range(num_buckets):
                    if duration2[t][i]>=buckets[num_buckets-k-1]:

                        duration_buckets[t][i]=num_buckets-k#jia le 1
                        break
        print "duration_buckets is",duration_buckets,"shape_duration_buckets is",np.shape(duration_buckets)
        max_yinsu_seq_len=np.max(yinsu_seq_len)
        for t in range(x):
            for i in range(max_yinsu_seq_len-len(yinsu[t])):
                yinsu[t].append(0)
                duration_buckets[t].append(0)
        #print "yinsu is ",yinsu,"yinsu_seq_len is ",yinsu_seq_len,"duration_buckets is ",duration_buckets
        print "voiced_target:",type(voiced_target),voiced_target
        %store yinsu
        %store yinsu_seq_len
        %store duration_buckets
        %store index_model4
        %store voiced_target
        
        #2,30
        coord.request_stop()
        coord.join(threads)
    

[array([[-317.97035132, -320.02675131, -320.49495957, ..., -315.87566673,
        -306.33839905, -257.33849693],
       [   6.82147203,    5.26051038,    5.43182948, ...,   12.73368223,
          23.06505468,   42.16068284],
       [   9.09845288,    6.68798967,    6.17807009, ...,   12.46905171,
          20.09967125,   25.53372145],
       ..., 
       [   5.00629905,    5.22212353,    5.08337967, ...,    4.68425215,
           3.78057042,   -1.52735552],
       [   4.70851533,    5.36507338,    4.96346314, ...,    4.61364601,
           5.37753316,    1.14885334],
       [   5.69843531,    5.74431946,    5.2694922 , ...,    4.50123533,
           8.12088201,    9.15983976]]), array([[-310.33822486, -309.71954486, -309.78283067, ..., -312.52741107,
        -307.24111342, -271.89655562],
       [   9.52933323,   10.77399375,   10.83669568, ...,    6.58462739,
          13.47175379,   36.30191069],
       [   9.90469798,   10.82214386,   10.77172059, ...,    6.78209292,
          12.44